In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
from scipy.io import arff #for reading the arff file

In [ ]:
# https://archive.ics.uci.edu/ml/datasets/Polish+companies+bankruptcy+data
# https://archive.ics.uci.edu/ml/machine-learning-databases/00365/
data = arff.loadarff('5year.arff')
# - 5thYear â€“ the data contains financial rates from 5th year of the forecasting period and corresponding class label
# that indicates bankruptcy status after 1 year.
df = pd.DataFrame(data[0])
# the length of data object is 2. The first item gives the dataset. The second item merely 
# states whether the variable is numeric or nominal

In [ ]:
type(data)

In [ ]:
len(data)

In [ ]:
data[1]

In [ ]:
df.head(2)
# data contains year 5 data and bankruptcy status after 1 year
# The class variable is nominal. It is a string. 

## Variable Definitions

In [ ]:
"""
X1 net profit / total assets
X2 total liabilities / total assets
X3 working capital / total assets
X4 current assets / short-term liabilities
X5 [(cash + short-term securities + receivables - short-term liabilities) / (operating expenses - depreciation)] * 365
X6 retained earnings / total assets
X7 EBIT / total assets
X8 book value of equity / total liabilities
X9 sales / total assets
X10 equity / total assets
X11 (gross profit + extraordinary items + financial expenses) / total assets
X12 gross profit / short-term liabilities
X13 (gross profit + depreciation) / sales
X14 (gross profit + interest) / total assets
X15 (total liabilities * 365) / (gross profit + depreciation)
X16 (gross profit + depreciation) / total liabilities
X17 total assets / total liabilities
X18 gross profit / total assets
X19 gross profit / sales
X20 (inventory * 365) / sales
X21 sales (n) / sales (n-1)
X22 profit on operating activities / total assets
X23 net profit / sales
X24 gross profit (in 3 years) / total assets
X25 (equity - share capital) / total assets
X26 (net profit + depreciation) / total liabilities
X27 profit on operating activities / financial expenses
X28 working capital / fixed assets
X29 logarithm of total assets
X30 (total liabilities - cash) / sales
X31 (gross profit + interest) / sales
X32 (current liabilities * 365) / cost of products sold
X33 operating expenses / short-term liabilities
X34 operating expenses / total liabilities
X35 profit on sales / total assets
X36 total sales / total assets
X37 (current assets - inventories) / long-term liabilities
X38 constant capital / total assets
X39 profit on sales / sales
X40 (current assets - inventory - receivables) / short-term liabilities
X41 total liabilities / ((profit on operating activities + depreciation) * (12/365))
X42 profit on operating activities / sales
X43 rotation receivables + inventory turnover in days
X44 (receivables * 365) / sales
X45 net profit / inventory
X46 (current assets - inventory) / short-term liabilities
X47 (inventory * 365) / cost of products sold
X48 EBITDA (profit on operating activities - depreciation) / total assets
X49 EBITDA (profit on operating activities - depreciation) / sales
X50 current assets / total liabilities
X51 short-term liabilities / total assets
X52 (short-term liabilities * 365) / cost of products sold)
X53 equity / fixed assets
X54 constant capital / fixed assets
X55 working capital
X56 (sales - cost of products sold) / sales
X57 (current assets - inventory - short-term liabilities) / (sales - gross profit - depreciation)
X58 total costs /total sales
X59 long-term liabilities / equity
X60 sales / inventory
X61 sales / receivables
X62 (short-term liabilities *365) / sales
X63 sales / short-term liabilities
X64 sales / fixed assets
"""

In [ ]:
df.info() # class stored as object

In [ ]:
df['class'].value_counts()

In [ ]:
# !pip install --upgrade numpy

In [ ]:
df['class'] = np.where(df['class']==b'0',0,1) #if

In [ ]:
# df['class']=df['class'].map(lambda x: 0 if x==b'0' else 1)

In [ ]:
df['class'] = df['class'].astype('category')

In [ ]:
df.info()

In [ ]:
print(df.shape)
print(df.dropna().shape)

In [ ]:
df.isnull().sum().sort_values(ascending=False)
# Attribute 37 has 2548 missing values. 

In [ ]:
# First drop Attr37 and then drop the missing values. 
df1 = df.drop('Attr37',axis=1).dropna(axis=0)
df1.shape

In [ ]:
df.shape
# original data has 5910 rows. We lose about 1000 rows of data

In [ ]:
df2 = df.drop(['Attr37','Attr27','Attr45'],axis=1).dropna(axis=0)
df2.shape

In [ ]:
len(df2.columns)-1


In [ ]:
df2.columns

In [ ]:
feature_columns = df2.columns[:df2.shape[1]-1]

In [ ]:
feature_columns

In [ ]:
X = df2[feature_columns]
y=df2['class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.75,random_state=13)

In [ ]:
# !pip install feature_engine

In [ ]:
from feature_engine.selection import DropCorrelatedFeatures

In [ ]:
tr = DropCorrelatedFeatures(threshold=0.8)

In [ ]:
X_train1 = tr.fit_transform(X_train)

In [ ]:
X_test.shape

In [ ]:
X_train.shape

In [ ]:
X_train1.shape

In [ ]:
y_train.shape

In [ ]:
final_features = X_train1.columns

In [ ]:
final_features

In [ ]:
X_test1 = X_test[final_features]

In [ ]:
y_train.value_counts()

In [ ]:
!pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
smote= SMOTE(random_state=13)
X_resampled, y_resampled = smote.fit_resample(X_train1, y_train)

In [ ]:
y_resampled.value_counts()

In [ ]:
X_resampled.shape

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(random_state=13,max_iter=1000)

In [ ]:
lr.fit(X_resampled,y_resampled)

In [ ]:
lr.coef_

In [ ]:
lr.intercept_

In [ ]:
print(f'The accuracy score on the training dataset is {lr.score(X_resampled,y_resampled):.2f}')
print(f'The accuracy score on the test dataset is {lr.score(X_test1,y_test):.2f}')

In [ ]:
y_pred = lr.predict(X_test1)

In [ ]:
y_prob = lr.predict_proba(X_test1)[:,1]

In [ ]:
X_test1.iloc[2].to_frame().T

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
ConfusionMatrixDisplay(confusion_matrix(y_test,y_pred),display_labels=["Non Bankrupt","Bankrupt"]).plot()

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test,y_pred).ravel()

In [ ]:
print(f'True Positive: {tp}')
print(f'True Negative: {tn}')
print(f'False Positive: {fp}')
print(f'False Negative: {fn}')


In [ ]:
from sklearn.metrics import precision_score, recall_score, roc_auc_score

In [ ]:
print(f'Precision Score is: {precision_score(y_test,y_pred):.2f}')
print(f'Recall Score is: {recall_score(y_test,y_pred):.3f}')

In [ ]:
lr.predict_proba(X_test1)

In [ ]:
print(f'Area under the ROC curve is:\
      {roc_auc_score(y_test,lr.predict_proba(X_test1)[:, 1]):.2f}')

In [ ]:
from sklearn.metrics import plot_roc_curve

In [ ]:
plot_roc_curve(lr,X_test1,y_test);

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_resampled,y_resampled)
y_pred = knn.predict(X_test1)
y_prob = knn.predict_proba(X_test1)
roc_auc_score(y_test,y_prob[:,1])

In [ ]:
from sklearn.naive_bayes import GaussianNB

In [ ]:
nb = GaussianNB()
nb.fit(X_resampled,y_resampled)
y_pred = nb.predict(X_test1)
y_prob = nb.predict_proba(X_test1)
roc_auc_score(y_test,y_prob[:,1])

In [ ]:
from sklearn.svm import NuSVC

In [ ]:
svm = NuSVC(kernel='rbf',random_state=13)
svm.fit(X_resampled,y_resampled)
y_pred = svm.predict(X_test1)
recall_score(y_test,y_pred)

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier(random_state=13,max_depth=5)
dt.fit(X_resampled,y_resampled)
y_pred = dt.predict(X_test1)
y_prob = dt.predict_proba(X_test1)
roc_auc_score(y_test,y_prob[:,1])

In [ ]:
pd.Series(dt.feature_importances_,index=final_features).sort_values(ascending=False)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier(n_estimators=500,random_state=13,max_depth=5)

In [ ]:
rf.fit(X_resampled,y_resampled)

In [ ]:
pd.Series(rf.feature_importances_,index=final_features).sort_values(ascending=False)

In [ ]:
y_pred = rf.predict(X_test1)
y_prob = rf.predict_proba(X_test1)
roc_auc_score(y_test,y_prob[:,1])